In [3]:
import base64
import requests
import os
from dotenv import load_dotenv

folder_path = '/Users/jiangwenchu/Desktop/自動推計/images'

# .envファイルから環境変数を読み込む
load_dotenv()

# 環境変数を取得
api_key = os.getenv('OPENAI_API_KEY')

# フォルダ内のすべてのファイルのパスを取得
# 全てJPEGに変換済
image_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
#print(image_paths)

# 画像をエンコードする関数
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 画像認識のリクエストを送信する関数

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

messages = []
#分類の配列
wood_chair = []
wood_desk = []
pp_chair = []
pp_desk = []

others = []

for image_path in image_paths:
    file_name = os.path.basename(image_path)
    base64_image = encode_image(image_path)
    message = {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "この画像が木製のイスなら1,木製の机なら2,ポリプロピレン製のいすなら3,ポリプロピレン製の机なら4,このうちどれでもなければ0と答えてください"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ]
    }
    messages.append(message)

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 300,
        "temperature": 1
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    #print(response.json())
    
    # APIの応答を取得
    response_data = response.json()

    # 質問に対する回答を抽出
    if 'choices' in response_data and len(response_data['choices']) > 0:
        # choicesの最初の要素からtextキーの内容を取得
        answer = response_data['choices'][0]['message']['content'].strip()
    else:
        print("回答が見つかりませんでした。")
    
    #4値分類
    if "1" in answer:
        wood_chair.append(file_name)                   
    elif "2" in answer:
        wood_desk.append(file_name)            
    elif "3" in answer:
        pp_chair.append(file_name)
    elif "4" in answer:
        pp_desk.append(file_name)
    elif "0" in answer:
        others.append(file_name)

#最終結果表示
wood_chair.sort()
wood_desk.sort()
pp_chair.sort()
pp_desk.sort()
others.sort()

print("木製、いす: ",wood_chair)
print("木製、つくえ: ", wood_desk)
print("pp製、いす: ", pp_chair)
print("pp製、つくえ: ", pp_desk)

print("others: ",others)
    




木製、いす:  ['image10.jpeg', 'image11.jpeg', 'image13.jpeg', 'image16.jpeg', 'image17.jpeg', 'image18.jpeg', 'image19.jpeg', 'image20.jpeg']
木製、つくえ:  ['image1.jpeg', 'image2.jpeg', 'image3.jpeg', 'image4.jpeg', 'image5.jpeg']
pp製、いす:  ['image12.jpeg', 'image14.jpeg', 'image15.jpeg', 'image6.jpeg', 'image8.jpeg']
pp製、つくえ:  []
others:  ['image7.jpeg', 'image9.jpeg']
